In [4]:
import tensorflow as tf

import read
import prepare_data
import input_builder

import importlib
import model

BATCH_SIZE = 1
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 128

importlib.reload(input_builder)
import os
os.environ['TFHUB_CACHE_DIR'] = '/home/djjindal/bert/script-learning'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
model_dir = 'output_triple'


# DATA

In [6]:
dataset = 'gw_extractions.pickle'
train_dataset = read.read_data_iterator(dataset)
features = list(prepare_data.tokenize_if_small_enough(train_dataset,sentences=False,no_context=False))
train_set = features[:int(0.8 * len(features))]
val_set = features[int(0.8 * len(features)):int(0.9*len(features))]
print(train_set[0])
len(train_set)

7874

# Model Objects

In [7]:
run_config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_summary_steps=0,
    save_checkpoints_steps=0,
    log_step_count_steps=100)

model_fn = model.model_fn_builder(
  num_labels=5,
  learning_rate=LEARNING_RATE,
  num_train_steps=1,
  num_warmup_steps=1)

train_test_input_fn = input_builder.input_fn_builder(
    features=train_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

val_test_input_fn = input_builder.input_fn_builder(
    features=val_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_eval_distribute': None, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9810fec518>, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_master': '', '_save_summary_steps': 0, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_protocol': None, '_global_id_in_cluster': 0, '_train_distribute': None, '_num_ps_replicas': 0, '_task_id': 0, '_device_fn': None, '_experimental_distribute': None, '_save_checkpoints_secs': None, '_tf_random_seed': None, '_task_type': 'worker', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': 0, '_model_dir': 'output_triple', '_evaluation_master': ''}


I0422 04:47:04.566318 140294139361024 estimator.py:201] Using config: {'_eval_distribute': None, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9810fec518>, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_master': '', '_save_summary_steps': 0, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_protocol': None, '_global_id_in_cluster': 0, '_train_distribute': None, '_num_ps_replicas': 0, '_task_id': 0, '_device_fn': None, '_experimental_distribute': None, '_save_checkpoints_secs': None, '_tf_random_seed': None, '_task_type': 'worker', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': 0, '_model_dir': 'output_triple', '_evaluation_master': ''}


# PREDICT FUNCTION

In [8]:
import numpy as np
def predict(sentenecs, triples, candidates, entity):
    e_dict = dict()
    check_dataset = []
    e_dict['sentences'] = sentenecs
    e_dict['triples'] = triples
    e_dict['candidates'] = candidates
    e_dict['correct'] = 4
    e_dict['entity'] = entity
    check_dataset = [e_dict]
    predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=False,no_context=False))

    predict_input_fn = input_builder.input_fn_builder(
        features=predict_set,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False,
        candidates=5)
    predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False,no_context=False)

    for (i, prediction) in enumerate(predictions):
        arr = prediction['probabilities'][0]
        prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
        total = np.sum(prob)
        max= np.max(prob)
        ec_dict = check_dataset[i]
        print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
        print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
        print("CANDIDATES", ec_dict['candidates'],"\n")
        print("ENTITY", ec_dict['entity'],"\n")
        print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")
        return prediction['labels']

def on_button_clicked(b):
    print(text1.value, text2.value, text3.value)
    print(predict([], text1.value.split('), ('), text2.value.split('), ('), text3.value))    

# DEMO

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
text1 = widgets.Text(description="Event Chain", width=200)
text2 = widgets.Text(description="Candidates", width=200)
text3 = widgets.Text(description="Entity", width=200)
button = widgets.Button(description="Predict")
button.on_click(on_button_clicked)
display(text1)
display(text2)
display(text3)
display(button)

# [('john','ordered',None),('john','ate','food')]
# [('john','left',None),('john','stays',None)]

# [('john','ordered',None),('john','paid',None),('john','ate','food')]
# [('john','left',None),('john','stays',None)]

# [(None, 'took', 'phone call'), (None, 'had', 'something'), (None, 'began', 'to squint')] 
# [(None, 'broadcast', '(the two previous record-holders'), ('the pennant', 'slammed', None), (None, "n't imagine", 'Maris'), (None, 'expressed', 'interest'), (None, 'turned', '68 _')] 

Text(value='', description='Event Chain')

Text(value='', description='Candidates')

Text(value='', description='Entity')

Button(description='Predict', style=ButtonStyle())

# Some Examples

In [10]:
dataset = list(read.read_data_iterator('gw_extractions.pickle'))
# train_data = dataset[:int(0.8 * len(features))]
val_data = dataset[int(0.8 * len(features)):int(0.9*len(features))]

for i, ec_dict in zip(range(10), val_data):
    check_dataset = [ec_dict]
    
    predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset, sentences=False))
    predict_input_fn = input_builder.input_fn_builder(
        features=predict_set,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False,
        candidates=5)

    predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
    for (i, prediction) in enumerate(predictions):
        arr = prediction['probabilities'][0]
        prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
        total = np.sum(prob)
        max= np.max(prob)
        print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
        print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
        print("CANDIDATES", ec_dict['candidates'],"\n")
        print("ENTITY", ec_dict['entity'],"\n")
        print("CORRECT", ec_dict['correct'] + 1,"\n")
        print("PREDICTION", prediction['labels'] + 1, "with Probability", np.max(prob),"\n","\n")



INFO:tensorflow:Could not find trained model in model_dir: output_triple, running initialization to predict.


I0422 04:47:06.796191 140294139361024 estimator.py:604] Could not find trained model in model_dir: output_triple, running initialization to predict.


INFO:tensorflow:Calling model_fn.


I0422 04:47:06.816890 140294139361024 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


I0422 04:48:58.972707 140294139361024 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0422 04:48:59.213999 140294139361024 session_manager.py:493] Done running local_init_op.


SENTENCE EVENT CHAIN ['Andreessen, 27, will join AOL as chief technology officer when\nthe $4.2 billion merger of the online titans closes next month,\nsources said.\n\n\n', "In addition to helping AOL and Netscape integrate their\ntechnologies as the combined company faces Microsoft, Yahoo, Excite\nand others in the fledgling Internet market, Andreessen's\nappointment is rife with symbolism.\n\n\n", "AOL officials hope the hiring averts a potential brain-drain at\nNetscape, where engineers historically dismissed AOL as the\n``McDonald's of cyberspace'' _ a high-tech lightweight that catered\nto the lowest common denominator.\n\n\n", "``It sends a reassuring signal to Netscape's engineers that AOL\nis serious about technology, and it maintains Andreessen's valuable\ncachet as an articulate spokesman for the company to the outside\nworld,'' said Jim Balderston of Zona Research in Redwood City.\n\n\n", "Still up in the air is whether anyone else from Netscape's\nexecutive offices will jo

In [12]:
dataset = list(read.read_data_iterator('gw_extractions.pickle'))
train_data = dataset[:int(0.8 * len(features))]
val_data = dataset[int(0.8 * len(features)):]

check_dataset = []

for i, ec_dict in zip(range(len(val_data)), val_data):
    check_dataset.append(ec_dict)
    
    
predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset,sentences=True,no_context=False))
predict_input_fn = input_builder.input_fn_builder(
    features=predict_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=5)

predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
predictions_list = list(predictions)
for (i, prediction) in enumerate(predictions):
    arr = prediction['probabilities'][0]
    prob = [np.exp(arr[0]),np.exp(arr[1]),np.exp(arr[2]),np.exp(arr[3]),np.exp(arr[4])]
    total = np.sum(prob)
    max= np.max(prob)
    ec_dict = check_dataset[i]
    print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
    print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
    print("CANDIDATES", ec_dict['candidates'],"\n")
    print("ENTITY", ec_dict['entity'],"\n")
    print(prob)
    print("CORRECT", ec_dict['correct'] + 1,"\n")
    print("PREDICTION", prediction['labels']+1, "with Probability", np.max(prob),"\n","\n")



INFO:tensorflow:Could not find trained model in model_dir: output_triple, running initialization to predict.


I0422 04:49:49.403486 140294139361024 estimator.py:604] Could not find trained model in model_dir: output_triple, running initialization to predict.


INFO:tensorflow:Calling model_fn.


I0422 04:50:10.102341 140294139361024 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0422 04:50:17.312193 140294139361024 monitored_session.py:222] Graph was finalized.


INFO:tensorflow:Running local_init_op.


I0422 04:50:18.917027 140294139361024 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0422 04:50:19.148870 140294139361024 session_manager.py:493] Done running local_init_op.


In [ ]:
bad = []
for i, c in enumerate(check_dataset):
    r = list(prepare_data.tokenize_if_small_enough([c],sentences=True,no_context=False))
    if not r:
        print(i)
        bad.append(i)
good_gt = [c for i, c in enumerate(check_dataset) if i not in bad]

In [ ]:
pred_df = pd.DataFrame.from_dict({'predictions':predictions_list, 'dataset':good_gt})

In [ ]:
pred_df['pred_label'] = pred_df.predictions.apply(lambda x: x['labels'] - 1)
pred_df['gt_label'] = pred_df.dataset.apply(lambda x: x['correct'])
pred_df['correct_pred'] = pred_df.apply(lambda s: s.pred_label == s.gt_label, axis=1)
print(pred_df.correct_pred.sum()/len(pred_df.index))

In [51]:
for ec_dict in check_dataset:
    print("SENTENCE EVENT CHAIN", ec_dict['sentences'],"\n")
    print("TRIPLE EVENT CHAIN", ec_dict['triples'],"\n")
    print("CANDIDATES", ec_dict['candidates'],"\n")
    print("ENTITY", ec_dict['entity'],"\n")
    print("CORRECT", ec_dict['correct'] + 1,"\n")
    print("PREDICTION", prediction['labels'], "with Probability", np.max(prob),"\n","\n")

SENTENCE EVENT CHAIN ["``We begin this session with a national spotlight on us,'' he told\nlawmakers.", 'Bush also took a a cue from Washington, where presidents often\nintroduce special guests during their State of the Union speeches.\n', 'Bush introduced ninth-grader Adam Gonzales of Lubbock as a student\nwho overcame years of social promotion that put him on an\neducational course to nowhere.\n\n\n'] 

TRIPLE EVENT CHAIN [(None, 'told', 'lawmakers'), (None, 'took', 'cue'), (None, 'introduced', 'Gonzales')] 

CANDIDATES [(None, 'introduced', 'Burgin'), (None, 'can offer', 'campaign-trail promises'), ('all the people', 'expanding', None), (None, 'echoing', 'light'), (None, 'will lay', 'chairman')] 

ENTITY Bush 

CORRECT 1 

PREDICTION 1 with Probability 0.8401629 
 

SENTENCE EVENT CHAIN ['\nAs a Vatican diplomat for almost 30 years, Justin\nRigali saw the fervor.', "When Rigali became archbishop of St. Louis in 1994, he wanted\nhis new flock to experience his friend's inspiration fi

TRIPLE EVENT CHAIN [(None, 'is', '69'), (None, 'will continue', 'to play'), (None, 'is', 'champion'), (None, 'will walk', 'hills'), (None, 'played', 'events'), (None, 'shrugs', 'advancing')] 

CANDIDATES [(None, 'shrugs', 'a shot'), (None, 'has graced', 'landscape'), ('a gilded view', 'made', None), ('the stuff', 'has not won', None), (None, 'guess', 'a gilded view')] 

ENTITY Arnold Palmer 

CORRECT 2 

PREDICTION 1 with Probability 0.8401629 
 

SENTENCE EVENT CHAIN ['\nThe New York Times said in an editorial Thursday, Jan. 7:\n\n\nFreshly re-elected, Gov. George Pataki gave a State of the State\naddress Wednesday that was a disappointing effort to make the\nincremental sound grand.', "In a long hour, however,\nPataki spoke only a few seconds on health care, not a word on how\nto allocate the tobacco settlement and one sentence, an\nafterthought perhaps, promising to clean up the state's abysmal\ncampaign finance system.\n\n\n", 'For felons, Pataki proposes an end to parole and the b